In [2]:
import numpy as np
import socialforce
from contextlib import contextmanager
import matplotlib as mpl
try:
    import matplotlib.pyplot as plt
    import matplotlib.animation as mpl_animation
except ImportError:
    plt = None
    mpl_animation = Non
%matplotlib inline
from numba import njit
from timeit import timeit

In [3]:
@contextmanager
def visualize(states, space):
    print('')
    with socialforce.show.animation(
            len(states),
            writer='imagemagick') as context:
        ax = context['ax']
        ax.set_xlabel('x [m]')
        ax.set_ylabel('y [m]')

        yield ax

        for s in space:
            ax.plot(s[:, 0], s[:, 1], '-o', color='black', markersize=2.5)

        actors = []
        for ped in range(states.shape[1]):
            speed = np.linalg.norm(states[0, ped, 2:4])
            radius = 0.2 + speed / 2.0 * 0.3
            p = plt.Circle(states[0, ped, 0:2], radius=radius,
                           facecolor='black' if states[0, ped, 4] > 0 else 'white',
                           edgecolor='black')
            actors.append(p)
            ax.add_patch(p)

        def update(i):
            for ped, p in enumerate(actors):
                # p.set_data(states[i:i+5, ped, 0], states[i:i+5, ped, 1])
                p.center = states[i, ped, 0:2]
                speed = np.linalg.norm(states[i, ped, 2:4])
                p.set_radius(0.2 + speed / 2.0 * 0.3)

        context['update_function'] = update

In [4]:
def test_separator():
    initial_state = np.array([
        [-10.0, -0.0, 1.0, 0.0, 10.0, 0.0],
    ])
    space = [
        np.array([(i, i) for i in np.linspace(-1, 4.0)]),
    ]
    s = socialforce.Simulator(initial_state, socialforce.PedSpacePotential(space))
    states = np.stack([s.step().state.copy() for _ in range(80)])

    # visualize
    with visualize(states, space, 'docs/separator.gif') as ax:
        ax.set_xlim(-10, 10)

In [5]:
def r_ab(state):
    """r_ab
    r_ab := r_a − r_b.
    """
    r = state[:, 0:2]
    r_a = np.expand_dims(r, 1)
    r_b = np.expand_dims(r, 0)
    return r_a - r_b


In [6]:
a = np.random.random((15,6))
r=a[:,0:2]; 
timeit('np.expand_dims(r, 1) ', setup='from __main__ import np, r')

3.903516436002974

In [7]:
r=a[:, 0:2]

In [8]:
np.expand_dims(r, 0)

array([[[0.93975664, 0.95845065],
        [0.80812148, 0.47425952],
        [0.50912247, 0.10253324],
        [0.93528894, 0.80475063],
        [0.13395655, 0.59670155],
        [0.5906764 , 0.62693926],
        [0.97491925, 0.49060199],
        [0.91463305, 0.87136243],
        [0.21889136, 0.42751392],
        [0.16115189, 0.79127455],
        [0.37068321, 0.72594091],
        [0.83708733, 0.24854019],
        [0.15199676, 0.45379353],
        [0.13972677, 0.20060956],
        [0.65781534, 0.97204912]]])

In [63]:
a = np.random.random((15,6))
b=[[1,2,3],[5,6]]


In [12]:
for i in b:
    print(a[i])

[[0.1575226  0.74302909 0.61273455 0.53950188 0.5704478  0.4693705 ]
 [0.59956391 0.19666366 0.78788385 0.20956496 0.69503992 0.97762946]
 [0.79268426 0.58102377 0.46123196 0.08876448 0.59370221 0.25486497]]
[[0.36040076 0.51324657 0.70009683 0.29982745 0.65001176 0.89407087]
 [0.1001721  0.18214999 0.62964004 0.23707038 0.27892567 0.79420435]]


In [15]:
a[i] for i in b

SyntaxError: invalid syntax (<ipython-input-15-c23b6853f86e>, line 1)

In [16]:
np.expand_dims([1,2,3,4,5],-1)

array([[1],
       [2],
       [3],
       [4],
       [5]])

In [17]:
def r_ab(state):
        """r_ab
        r_ab := r_a − r_b.
        """
        r = state[:, 0:2]
        r_a = np.expand_dims(r, 1)
        r_b = np.expand_dims(r, 0)
        return r_a - r_b

In [31]:
r_ab(a)[1,2]

array([-0.44204132,  0.54636544])

In [34]:
from scipy.spatial import distance_matrix
distance_matrix(a,a)

array([[0.        , 0.56722521, 1.03060711, 0.90543849, 0.65353686,
        0.78068562, 0.7898188 , 0.94297426, 1.07174323, 0.75136439,
        0.77537355, 0.63041696, 0.88048003, 1.18841522, 0.78985669],
       [0.56722521, 0.        , 0.95252387, 0.83806127, 0.51728743,
        0.58799353, 0.77467879, 1.07950616, 0.6383511 , 0.57132917,
        0.79150664, 0.67287851, 0.95833494, 0.86355125, 1.05693606],
       [1.03060711, 0.95252387, 0.        , 0.91595864, 1.19872526,
        0.42695105, 0.69440357, 0.96338586, 1.36013589, 0.71876234,
        1.016986  , 0.9400819 , 1.2867366 , 1.28411991, 0.99712962],
       [0.90543849, 0.83806127, 0.91595864, 0.        , 0.66820665,
        0.83953662, 1.03874951, 0.55521142, 1.05876025, 0.71952653,
        1.27934395, 1.0096857 , 0.98341118, 0.97646534, 0.66418884],
       [0.65353686, 0.51728743, 1.19872526, 0.66820665, 0.        ,
        0.91188829, 0.93909714, 0.98995636, 0.75050989, 0.66519697,
        0.98792092, 0.77423619, 1.0334549 , 

In [91]:
from numba import njit

@njit
def speeds(state):
    """Return the speeds corresponding to a given state."""
    speeds = []
    for s in state[:, 2:4]:
        speeds.append(np.linalg.norm(s))
    return np.array(speeds)
def speeds2(state):
    """Return the speeds corresponding to a given state."""
    return np.linalg.norm(state[:, 2:4], axis=-1)

In [92]:
from timeit import timeit

In [93]:
timeit("desired_directions(a)", setup="from __main__ import desired_directions, a")

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mInvalid use of Function(<built-in function setitem>) with argument(s) of type(s): (array(float64, 2d, C), array(bool, 1d, C), list(int64))
 * parameterized
[1mIn definition 0:[0m
[1m    All templates rejected with literals.[0m
[1mIn definition 1:[0m
[1m    All templates rejected without literals.[0m
[1mIn definition 2:[0m
[1m    All templates rejected with literals.[0m
[1mIn definition 3:[0m
[1m    All templates rejected without literals.[0m
[1mIn definition 4:[0m
[1m    All templates rejected with literals.[0m
[1mIn definition 5:[0m
[1m    All templates rejected without literals.[0m
[1mIn definition 6:[0m
[1m    All templates rejected with literals.[0m
[1mIn definition 7:[0m
[1m    All templates rejected without literals.[0m
[1mIn definition 8:[0m
[1m    All templates rejected with literals.[0m
[1mIn definition 9:[0m
[1m    All templates rejected without literals.[0m

In [77]:
b = a[:, 4:6] - a[:, 0:2]
c=[]
for i in b:
    c.append(np.linalg.norm(i))

In [79]:
c = np.asarray(c)
c

array([1.04014516, 0.44271955, 0.7085655 , 0.75631648, 0.83731874,
       0.88986989, 0.70479232, 0.46156009, 0.4002257 , 0.78521197,
       0.29882979, 0.5646522 , 0.47329184, 0.59147457, 0.68845587])

In [73]:
np.linalg.norm(b, axis=-1)

array([1.04014516, 0.44271955, 0.7085655 , 0.75631648, 0.83731874,
       0.88986989, 0.70479232, 0.46156009, 0.4002257 , 0.78521197,
       0.29882979, 0.5646522 , 0.47329184, 0.59147457, 0.68845587])

In [87]:
np.expand_dims(c, -1).shape

(15, 1)

In [86]:
c.shape

(15,)

In [172]:
from abc import ABC, abstractmethod

class A(ABC):
    def __init__(self):
        self.name=type(self).__name__

    @abstractmethod
    def aa(self):
        pass

class B(A):
    def aa(self):
        return np.random.random((3,3))

In [115]:
b=B()
b.aa(1)


B


In [116]:
import re

name = 'CamelCaseName'
name = re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower()
print(name)  # camel_case_name

camel_case_name


In [120]:
k={1:2}
k.get(3) is None

True

In [134]:
from socialforce.potentials import PedPedPotential
from socialforce import stateutils

In [143]:
V = PedPedPotential(1)
a = np.random.random((15,7))
V.grad_r_ab(a).shape


(15, 15, 2)

In [144]:
vel = a[:, 2:4]
tau = a[:, 6:7]
e = stateutils.desired_directions(a)
F0 = 1.0 / tau * (np.expand_dims(stateutils.speeds(a), -1) * e - vel)

In [147]:
F0.shape

(15, 2)

In [174]:
bl = [B(), B(), B()]
sum(map(lambda x: x.aa(), bl))

array([[2.23380899, 1.92670594, 1.46995999],
       [2.383146  , 0.63375966, 1.42286419],
       [0.59535968, 1.8066675 , 2.06413655]])

In [158]:
space = [
        np.array([(0.0, y) for y in np.linspace(-10, -0.7, 1000)]),
        np.array([(0.0, y) for y in np.linspace(0.7, 10, 1000)]),
    ]

In [164]:
import toml
f = toml.loads('''title = "Social Force Config File"

time_step = 0.4
max_speed_multiplier = 1.3
tau = 0.5

[goal_attractive_force]
factor = 1.0

[ped_repulsive_force]
factor = 1.0
v0 = 10
sigma = 0.3
# fov params
fov_phi = 100.0
fov_factor = 0.5 # out of view factor

[space_repulsive_force]
factor = 1.0
u0 = 10
r = 0.2

[group_coherence_force]

[group_repulsive_force]

[group_gaze_force]''')

In [165]:
f

{'title': 'Social Force Config File',
 'time_step': 0.4,
 'max_speed_multiplier': 1.3,
 'tau': 0.5,
 'goal_attractive_force': {'factor': 1.0},
 'ped_repulsive_force': {'factor': 1.0,
  'v0': 10,
  'sigma': 0.3,
  'fov_phi': 100.0,
  'fov_factor': 0.5},
 'space_repulsive_force': {'factor': 1.0, 'u0': 10, 'r': 0.2},
 'group_coherence_force': {},
 'group_repulsive_force': {},
 'group_gaze_force': {}}

In [183]:
from scipy.spatial import distance_matrix
b = a[[2,3,4]]
distance_matrix(b[:, 0:2],b[:, 0:2])

array([[0.        , 0.10110868, 0.36682945],
       [0.10110868, 0.        , 0.39915365],
       [0.36682945, 0.39915365, 0.        ]])

In [181]:
b[:,:2]

array([[0.13139352, 0.33601537],
       [0.03511724, 0.30513101],
       [0.30972286, 0.01544948]])

In [210]:
from scipy.spatial.distance import cdist
k=cdist([[0.13139352, 0.33601537],
       [0.03511724, 0.30513101],
       [0.30972286, 0.01544948]], [[0.13139352, 0.33601537],
       [0.03511724, 0.30513101],
       [0.30972286, 0.01544948]])

In [213]:
k[k<0.2]

array([0.        , 0.10110868, 0.10110868, 0.        , 0.        ])

In [222]:
l=np.expand_dims(k, axis=-1)

In [234]:
l[l>0.2]=0
np.sum(l, axis=1)

array([[0.10110868],
       [0.10110868],
       [0.        ]])

In [230]:
l

array([[[0.        ],
        [0.10110868],
        [0.        ]],

       [[0.10110868],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ]]])

In [208]:
timeit("group_center(b)", setup="from __main__ import group_center, b")

0.8395808530040085

In [238]:
k=np.zeros((a.shape[0],1))

In [239]:
k[[2,3,4]] = np.sum(l, axis=1)

In [240]:
k

array([[0.        ],
       [0.        ],
       [0.10110868],
       [0.10110868],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ]])

In [4]:
import numpy as np
state = np.array(
        [
            [
                0.20641395,
                0.37025059,
                0.83481168,
                0.81327761,
                0.40032738,
                0.02289462,
                0.07277386,
            ],
            [
                0.86297966,
                0.12240028,
                0.74459341,
                0.7000413,
                0.85403137,
                0.39727004,
                0.38040942,
            ],
            [
                0.13139352,
                0.33601537,
                0.96760778,
                0.95073568,
                0.0067437,
                0.81560067,
                0.95832751,
            ],
            [
                0.03511724,
                0.30513101,
                0.74446203,
                0.26100916,
                0.90492352,
                0.17652219,
                0.83667939,
            ],
            [
                0.30972286,
                0.01544948,
                0.51499794,
                0.53418732,
                0.28598274,
                0.6405337,
                0.47180045,
            ],
            [
                0.26194976,
                0.46709103,
                0.88775726,
                0.36179478,
                0.6046481,
                0.56133123,
                0.28768578,
            ],
        ]
    )
groups = [[1, 3, 4], [0, 2]]

In [8]:
from socialforce.forces import *
import toml
f = GroupRepulsiveForce()
f.config={'factor': 1.0, 'threshold': 0.5}
f.set_state(state, groups=groups)
f.get_force()


array([[0.08246281],
       [0.        ],
       [0.08246281],
       [0.39915365],
       [0.39915365],
       [0.        ]])

In [10]:
state[:,:2] - [0.5,1]

array([[-0.29358605, -0.62974941],
       [ 0.36297966, -0.87759972],
       [-0.36860648, -0.66398463],
       [-0.46488276, -0.69486899],
       [-0.19027714, -0.98455052],
       [-0.23805024, -0.53290897]])

In [16]:
com = stateutils.group_center([state[[2,3,4]][:, :2]])

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mInvalid use of Function(<built-in function getitem>) with argument(s) of type(s): (reflected list(array(float64, 2d, A)), UniTuple(slice<a:b> x 2))
 * parameterized
[1mIn definition 0:[0m
[1m    All templates rejected with literals.[0m
[1mIn definition 1:[0m
[1m    All templates rejected without literals.[0m
[1mIn definition 2:[0m
[1m    All templates rejected with literals.[0m
[1mIn definition 3:[0m
[1m    All templates rejected without literals.[0m
[1mIn definition 4:[0m
[1m    All templates rejected with literals.[0m
[1mIn definition 5:[0m
[1m    All templates rejected without literals.[0m
[1mIn definition 6:[0m
[1m    All templates rejected with literals.[0m
[1mIn definition 7:[0m
[1m    All templates rejected without literals.[0m
[1mIn definition 8:[0m
[1m    All templates rejected with literals.[0m
[1mIn definition 9:[0m
[1m    All templates rejected without 

In [22]:
np.linalg.norm(com - state[[2,3,4]][:,:2], axis=1)

array([0.12030054, 0.15074974, 0.2533228 ])

In [20]:
(com - state[[2,3,4]][:,:2]).shape

(3, 2)

In [67]:
from numba import njit,jit
from timeit import timeit
# @jit
# def normalize(mat):
#     norm_factors = np.linalg.norm(mat, axis=-1)
#     directions = mat / np.expand_dims(norm_factors, -1)
#     return directions
@njit
def normalize(mat):
    norm_factors = []
    for line in mat:
        norm_factors.append(np.linalg.norm(line))
    normalized = mat / np.expand_dims(np.array(norm_factors), -1)
    return normalized, norm_factors

In [68]:
timeit("normalize(state[:,:2])", setup="from __main__ import normalize, state")

2.34487513199565

In [64]:
normalize(state[:,:2])

array([[0.48693883, 0.87343607],
       [0.99009075, 0.14042902],
       [0.3641812 , 0.93132811],
       [0.11433434, 0.99344233],
       [0.99875823, 0.04981968],
       [0.48914172, 0.87220432]])

In [16]:
from socialforce.stateutils import normalize, vec_diff
m = state[:4, :2]
v, d = normalize(vec_diff(m))

In [25]:
v

array([[[ 0.        ,  0.        ],
        [-0.490452  ,  0.1851432 ],
        [ 0.0970231 ,  0.04427604],
        [ 0.19616163,  0.07457214]],

       [[ 0.89941222, -0.33952367],
        [ 0.        ,  0.        ],
        [ 0.94615237, -0.27626607],
        [ 0.94803247, -0.20925538]],

       [[-0.10276853, -0.04689793],
        [-0.54649196,  0.15956963],
        [ 0.        ,  0.        ],
        [ 0.11025146,  0.03536744]],

       [[-0.23465489, -0.08920561],
        [-0.61840997,  0.13649914],
        [-0.12451306, -0.03994241],
        [ 0.        ,  0.        ]]])

In [24]:
d

[0.7299942092078088,
 1.3386951485552256,
 0.7732223276037036,
 0.8732426893332906]

In [36]:
[normalize(l) for l in vec_diff(m)]

[(array([[        nan,         nan],
         [-0.93555951,  0.35316909],
         [ 0.90974864,  0.41515951],
         [ 0.93473496,  0.35534569]]),
  [0.0, 0.7017893613577368, 0.08246281104736425, 0.18325698501339727]),
 (array([[ 0.93555951, -0.35316909],
         [        nan,         nan],
         [ 0.95991679, -0.28028512],
         [ 0.97649547, -0.2155379 ]]),
  [0.7017893613577368, 0.0, 0.7621349532174783, 0.8477893052725951]),
 (array([[-0.90974864, -0.41515951],
         [-0.95991679,  0.28028512],
         [        nan,         nan],
         [ 0.95220586,  0.30545705]]),
  [0.08246281104736425, 0.7621349532174783, 0.0, 0.10110868302597954]),
 (array([[-0.93473496, -0.35534569],
         [-0.97649547,  0.2155379 ],
         [-0.95220586, -0.30545705],
         [        nan,         nan]]),
  [0.18325698501339727, 0.8477893052725951, 0.10110868302597954, 0.0])]

In [32]:
0.93555951**2+ 0.35316909**2

1.0000000028828684

In [40]:
f_ab=np.zeros((6, 6, 2))
np.sum(f_ab, axis=1).shape

(6, 2)

In [44]:
forces = np.zeros((state.shape[0], 2))
forces[[1,2,3],:]

array([[0., 0.],
       [0., 0.],
       [0., 0.]])

In [57]:
@njit
def normalize(mat):
    norm_factors = []
    for line in mat:
        norm_factors.append(np.linalg.norm(line))
    norm_factors = np.array(norm_factors)
    normalized = mat / np.expand_dims(norm_factors, -1)
    return normalized, norm_factors

In [63]:
vectors, norms =normalize(vec_diff(state[:, :2])[0])
vectors = np.nan_to_num(vectors)
vectors[norms>0.2] = [0,0]

In [11]:
import numpy as np
import timeit
from numba import njit, jit

In [9]:
%timeit np.sum([1,2])

5.88 µs ± 22.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [11]:
%load_ext cython

In [65]:
%%cython
import numpy as np
cimport numpy as np
def normalize2(double[:,:] mat):
    cdef double[:] norm_factors = np.linalg.norm(mat, axis=-1)
    normalized = mat / np.expand_dims(norm_factors, -1)
    return normalized, norm_factors



In [15]:
state = np.random.random((15,6))

In [54]:
from socialforce.stateutils import *

In [66]:
%timeit normalize2(state)

24.2 µs ± 91.2 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [51]:
state[:, :2]

array([[0.8684355 , 0.51035531],
       [0.74282832, 0.50520676],
       [0.81173266, 0.17328422],
       [0.82101978, 0.25224042],
       [0.7193332 , 0.42492837],
       [0.69941293, 0.36886388],
       [0.39712639, 0.73841454],
       [0.78735899, 0.98971974],
       [0.1424415 , 0.02370078],
       [0.26892978, 0.03206733],
       [0.80114236, 0.96106836],
       [0.62269205, 0.4986398 ],
       [0.27041573, 0.4958112 ],
       [0.07100506, 0.44698214],
       [0.5383372 , 0.92525308]])

In [36]:
state


array([[0.79443155, 0.34942958, 0.0369333 , 0.55426836, 0.55716464,
        0.80173158, 0.70128969],
       [0.01240691, 0.70700964, 0.57016233, 0.98013914, 0.80394197,
        0.16704183, 0.36519203],
       [0.73017366, 0.64449567, 0.40819632, 0.21381814, 0.2967929 ,
        0.10872888, 0.23028855],
       [0.18295271, 0.85908636, 0.92276844, 0.94251612, 0.78974431,
        0.45430138, 0.91943352],
       [0.53329596, 0.29978034, 0.88640945, 0.44236774, 0.44278228,
        0.12198758, 0.51128694],
       [0.4120075 , 0.13384818, 0.23175562, 0.45509601, 0.14269477,
        0.50164887, 0.06625182],
       [0.54656168, 0.64144692, 0.39937917, 0.9466161 , 0.18935002,
        0.80727991, 0.28783943],
       [0.9747244 , 0.96133071, 0.26456555, 0.13023425, 0.23209924,
        0.57321796, 0.26774286],
       [0.42882787, 0.85950015, 0.76398687, 0.23927456, 0.49231733,
        0.08383806, 0.72249575],
       [0.46652611, 0.78587203, 0.0697163 , 0.6821024 , 0.94478028,
        0.84199639, 0.6

In [1]:
from socialforce.forces import *
import numpy as np
state = np.zeros((5,7))
state[:,:2] = np.array([[1,1],[1,1.1],[3,3],[3,3.01],[3,4]])



In [94]:
class GroupCoherenceForceAlt(Force):
    def get_force(self):
        forces = np.zeros((self.state.shape[0], 2))
        for group in self.groups:
            threshold = (len(group) - 1) / 2
            member_states = self.state[group, :]
            member_pos = member_states[:, 0:2]
            com = stateutils.group_center(member_states)
            force_vec = com - member_pos
            vectors, norms = normalize(force_vec)
            softened_factor = (np.tanh(norms - threshold) + 1) / 2
            forces[group, :] += (force_vec.T * softened_factor).T
        return forces * self.factor

class GroupCoherenceForce(Force):
    def get_force(self):
        forces = np.zeros((self.state.shape[0], 2))
        for group in self.groups:
            threshold = (len(group) - 1) / 2
            member_states = self.state[group, :]
            member_pos = member_states[:, 0:2]
            com = stateutils.group_center(member_states)
            force_vec = com - member_pos
            vectors, norms = normalize(force_vec)
            print(vectors, norms)
            vectors[norms < threshold] = [0, 0]
            forces[group, :] += vectors
        return forces * self.factor

class GroupGazeForce(Force):
    def get_force(self):
        forces = np.zeros((self.state.shape[0], 2))
        vision_angle = self.config.get('fov_phi') or 100.0
        walking_directions = stateutils.desired_directions(self.state)
        for group in self.groups:
            group_size = len(group)
            # 1-agent groups don't need to compute this
            if group_size <= 1:
                continue
            member_states = self.state[group, :]
            member_pos = member_states[:, 0:2]
            member_directions = walking_directions[group, :]
            # use center of mass without the current agent
            relative_com = np.array([stateutils.group_center(member_pos[np.arange(group_size)!=i, :]) - member_pos[i, :] for i in range(group_size)])

            com_directions, com_dists = normalize(relative_com)
            # angle between walking direction and center of mass
            com_angles = np.degrees([np.arccos(np.dot(d, c)) for d, c in zip(member_directions, com_directions)] )
            rotation = np.radians([a - vision_angle if a > vision_angle else 0.0 for a in com_angles])
            force = -np.expand_dims(rotation, -1) * member_directions
            forces[group, :] += force
        return forces * self.factor

f = GroupGazeForce()
f.set_state(state, groups=[[0,1,3], [2,4]])
f.get_force()

array([[0.96838684, 0.96838684],
       [0.87370295, 0.96107324],
       [0.43194695, 0.43194695],
       [0.        , 0.        ],
       [0.        , 0.        ]])

In [48]:
%timeit f.get_force()

303 µs ± 1.88 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [61]:
 np.array([[
            [0.0, 0.0],
            [1.0, 1.0],
        ], [
            [-1.0, 1.0],
            [0.0, 0.0],
        ],[
            [-1.0, 1.0],
            [0.0, 0.0],
        ]
        ]).shape

(3, 2, 2)

In [54]:
np.array([ 0.66666667,  0.7       ]) / np.array([ 0.68965517,  0.72413793]) 

array([0.96666667, 0.96666667])

In [23]:
%timeit dist(state)

1.22 µs ± 1.03 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [39]:
%timeit normalize(state)

3.94 µs ± 2.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [37]:
@njit
def distances_numba_array(cluster):
    # Original: diff = cluster[:, np.newaxis, :] - cluster[np.newaxis, :, :]
    # Since np.newaxis is not supported, we use reshape to do this
    diff = (cluster.reshape(cluster.shape[0], 1, cluster.shape[1]) -
            cluster.reshape(1, cluster.shape[0], cluster.shape[1]))
    mat = (diff * diff)
    # Original: mat = mat.sum(-1)
    # Since axis argument is not supported, we write the loop out
    out = np.empty(mat.shape[:2], dtype=mat.dtype)
    for i in np.ndindex(out.shape):
        out[i] = mat[i].sum()

    return np.sqrt(out)

In [44]:
@jit
def desired_directions(state):
    """Given the current state and destination, compute desired direction."""
    destination_vectors = state[:, 4:6] - state[:, 0:2]
    directions, norm_factors = normalize(destination_vectors)
    directions[norm_factors == 0] = [0, 0]
    return directions

@njit
def desired_directions2(state):
    """Given the current state and destination, compute desired direction."""
    destination_vectors = state[:, 4:6] - state[:, 0:2]
    directions, norm_factors = normalize(destination_vectors)
    for i in range(norm_factors.shape[0]):
        if norm_factors[i] == 0:
            directions[i] = [0,0]
    return directions

In [45]:
%timeit desired_directions(state)

14.9 µs ± 29.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [46]:
%timeit desired_directions2(state)

5.51 µs ± 3.6 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
k=np.dot([0.5,0.5], [[1,1,1],[2,2,2]])

In [32]:
k

array([1.5, 1.5, 1.5])

In [58]:
a = np.ones((3,2))
b = np.array([1,1.5,1])

In [73]:
import numpy as np
state = np.random.random((5,7))

In [33]:
from itertools import combinations
state[np.arange(4).,:].shape


IndexError: boolean index did not match indexed array along dimension 0; dimension is 5 but corresponding boolean dimension is 4

In [3]:
import numpy as np
import socialforce
import matplotlib.cm as cm

initial_state = np.array(
    [
        [0.0, 0.0, 0.5, 0.5, 10.0, 10.0],
        [0.0, 1.0, 0.5, 0.5, 10.0, 10.0],
        [10.0, 0.3, -0.5, 0.5, 0.0, 10.0],
        [11.0, 0.3, -0.5, 0.5, 0.0, 10.0],
        [12.0, 0.3, -0.5, 0.5, 0.0, 10.0],
    ]
)
groups = [[0, 1], [2, 3, 4]]
colors = cm.rainbow(np.linspace(0, 1, len(groups)))
s = socialforce.Simulator(initial_state, groups=groups)
states = np.stack([s.step().state.copy() for _ in range(100)])

with socialforce.show.canvas("docs/group_crossing.png") as ax:
    ax.set_xlabel("x [m]")
    ax.set_ylabel("y [m]")

    for i, group in enumerate(groups):
        color = colors[i]
        for ped in group:
            x = states[:, ped, 0]
            y = states[:, ped, 1]
            ax.plot(
                x, y, "-o", label="ped {}".format(ped), markersize=2.5, color=color
            )
    ax.legend()


None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
